![Public Domain Lisp Logo Set By Conrad Barski, M.D.](http://www.lisperati.com/lisplogo_fancy_128.png)

In [ ]:
(require 'asdf)

In [ ]:
(require 'hdf5-cffi)

In [ ]:
(use-package "HDF5")

In [ ]:
(require 'alcove)

In [ ]:
(use-package "ALCOVE")

In [ ]:
(defparameter *file-name* "superblock.h5")

We can use the Lisp [Common Foreign Function Interface](https://common-lisp.net/project/cffi/) (CFFI) to call the HDF5 library and create an "empty" HDF5 file.

In [ ]:
(let ((fcpl (h5pcreate +H5P-FILE-CREATE+)))
     (h5fclose (h5fcreate *file-name* +H5F-ACC-TRUNC+ +H5P-DEFAULT+ +H5P-DEFAULT+))  ; create and close the file
     (h5pclose fcpl))

We use Lisp's [``OPEN``](http://clhs.lisp.se/Body/f_open.htm) function to create a file stream that is connected to the HDF5 file.

In [ ]:
(defparameter *input-stream*
    (open *file-name* :direction :input :element-type '(unsigned-byte 8)))

For the moment we assume that the superblock is located at the beginning of the HDF5 file. (We'll discuss so-called user blocks in the next installment.) We use the function ``READ-SUPERBLOCK`` to read the HDF5 file's superblock and return it in the form of an [association list](http://clhs.lisp.se/Body/26_glo_a.htm#association_list) ("a list of key/value pairs"). 

In [ ]:
(documentation 'read-superblock 'function)

In [ ]:
(read-superblock *input-stream*)

The meaning of the different fields is decribed in the [HDF5 File Format Specification](https://www.hdfgroup.org/HDF5/doc/H5.format.html).

18,446,744,073,709,551,615 is the *undefined address* $2^{64}-1$.

This is a version 0 superblock. In the next section, we'll look at  a version 2 superblock.

>Version 0 is the default format, while version 1 is basically the same as version 0 with additional information when a non-default B-tree 'K' value is stored. Version 2 is the latest format, with some fields eliminated or compressed and with superblock extension and checksum support.

In [ ]:
(close *input-stream*)

#### Superblock Version 2

In [ ]:
(setf *file-name* "superblock.v2.h5")

We create an HDF5 file with a version 2 superblock by specifying a non-default *file access property list*, which instructs the library to use the latest version for creating objects in the file.

In [ ]:
(let ((fapl (h5pcreate +H5P-FILE-ACCESS+)))
     (h5pset-libver-bounds fapl :H5F-LIBVER-LATEST :H5F-LIBVER-LATEST)
     (h5fclose (h5fcreate *file-name* +H5F-ACC-TRUNC+ +H5P-DEFAULT+ fapl))
     (h5pclose fapl))

We read the superblock from a file stream as we did previously.

In [ ]:
(setf *input-stream* (open *file-name* :direction :input :element-type '(unsigned-byte 8)))

In [ ]:
(read-superblock *input-stream*)

The version 2 superblock appears a little more clearly arranged. The main new ingredient is the superblock checksum.

>All checksums used in the format are computed with the [Jenkins’ lookup](http://www.burtleburtle.net/bob/hash/doobs.html) algorithm.

In [ ]:
(close *input-stream*)